In [1]:
"""
Vanilla Char-RNN using TensorFlow by Vinh Khuc (@knvinh).
Adapted from Karpathy's min-char-rnn.py
https://gist.github.com/karpathy/d4dee566867f8291f086
Requires tensorflow>=1.0
BSD License
"""
import random
import numpy as np
import tensorflow as tf

seed_value = 42
tf.set_random_seed(seed_value)
random.seed(seed_value)

def one_hot(v):
    return np.eye(vocab_size)[v]

# Data I/O
data = open('Hierarchy.txt', 'r').read()  # Use this source file as input for RNN
chars = sorted(list(set(data)))
data_size, vocab_size = len(data), len(chars)
print('Data has %d characters, %d unique.' % (data_size, vocab_size))
char_to_ix = {ch: i for i, ch in enumerate(chars)}
ix_to_char = {i: ch for i, ch in enumerate(chars)}

# Hyper-parameters
hidden_size   = 100  # hidden layer's size
seq_length    = 25   # number of steps to unroll
learning_rate = 1e-1

inputs     = tf.placeholder(shape=[None, vocab_size], dtype=tf.float32, name="inputs")
targets    = tf.placeholder(shape=[None, vocab_size], dtype=tf.float32, name="targets")
init_state = tf.placeholder(shape=[1, hidden_size], dtype=tf.float32, name="state")

initializer = tf.random_normal_initializer(stddev=0.1)

with tf.variable_scope("RNN") as scope:
    hs_t = init_state
    ys = []
    for t, xs_t in enumerate(tf.split(inputs, seq_length, axis=0)):
        if t > 0: scope.reuse_variables()  # Reuse variables
        Wxh = tf.get_variable("Wxh", [vocab_size, hidden_size], initializer=initializer)
        Whh = tf.get_variable("Whh", [hidden_size, hidden_size], initializer=initializer)
        Why = tf.get_variable("Why", [hidden_size, vocab_size], initializer=initializer)
        bh  = tf.get_variable("bh", [hidden_size], initializer=initializer)
        by  = tf.get_variable("by", [vocab_size], initializer=initializer)

        hs_t = tf.tanh(tf.matmul(xs_t, Wxh) + tf.matmul(hs_t, Whh) + bh)
        ys_t = tf.matmul(hs_t, Why) + by
        ys.append(ys_t)

hprev = hs_t
output_softmax = tf.nn.softmax(ys[-1])  # Get softmax for sampling

outputs = tf.concat(ys, axis=0)
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=targets, logits=outputs))

# Minimizer
minimizer = tf.train.AdamOptimizer()
grads_and_vars = minimizer.compute_gradients(loss)

# Gradient clipping
grad_clipping = tf.constant(5.0, name="grad_clipping")
clipped_grads_and_vars = []
for grad, var in grads_and_vars:
    clipped_grad = tf.clip_by_value(grad, -grad_clipping, grad_clipping)
    clipped_grads_and_vars.append((clipped_grad, var))

# Gradient updates
updates = minimizer.apply_gradients(clipped_grads_and_vars)

# Session
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

# Initial values
n, p = 0, 0
hprev_val = np.zeros([1, hidden_size])

while True:
    # Initialize
    if p + seq_length + 1 >= len(data) or n == 0:
        hprev_val = np.zeros([1, hidden_size])
        p = 0  # reset

    # Prepare inputs
    input_vals  = [char_to_ix[ch] for ch in data[p:p + seq_length]]
    target_vals = [char_to_ix[ch] for ch in data[p + 1:p + seq_length + 1]]

    input_vals  = one_hot(input_vals)
    target_vals = one_hot(target_vals)

    hprev_val, loss_val, _ = sess.run([hprev, loss, updates],
                                      feed_dict={inputs: input_vals,
                                                 targets: target_vals,
                                                 init_state: hprev_val})
    if n % 500 == 0:
        # Progress
        print('iter: %d, p: %d, loss: %f' % (n, p, loss_val))

        # Do sampling
        sample_length = 200
        start_ix      = random.randint(0, len(data) - seq_length)
        sample_seq_ix = [char_to_ix[ch] for ch in data[start_ix:start_ix + seq_length]]
        ixes          = []
        sample_prev_state_val = np.copy(hprev_val)

        for t in range(sample_length):
            sample_input_vals = one_hot(sample_seq_ix)
            sample_output_softmax_val, sample_prev_state_val = \
                sess.run([output_softmax, hprev],
                         feed_dict={inputs: sample_input_vals, init_state: sample_prev_state_val})

            ix = np.random.choice(range(vocab_size), p=sample_output_softmax_val.ravel())
            ixes.append(ix)
            sample_seq_ix = sample_seq_ix[1:] + [ix]

        txt = ''.join(ix_to_char[ix] for ix in ixes)
        print('----\n %s \n----\n' % (txt,))

    p += seq_length
    n += 1

Data has 7317 characters, 77 unique.
iter: 0, p: 0, loss: 4.384295
----
 	3^FW1Q9lSfAmR
ipoNv+dsm VP<CN:/tpN)8)8nwF/(o8n8vSRjPLkfLQcAaB4RZ9+Tz	eodCfq).yosWvnwhRmt9SNNJS+)WPz/)SnfJE)bkUJP :+MwEb)o5f,1UQKH2^fmhmZvK0Q2-0a0is*Ub*	a)cy&jWUhi/GyGiJraKhDiCtJtvFmEjKrfE<E#^oxD+t9Ru 
----

iter: 500, p: 5200, loss: 3.211074
----
 eveLT,i
Cint,(p  p               ), One  APeror    Ap u  *l QleeBrotneew2ip
l TaaLC
y   Okeanb,ocp>f        te oap F  oeeeO9R* Z  o     lelorSPls
  e  V*al,Pu
ciPrd,oNnna              D  Me aMber opD  
----

iter: 1000, p: 3100, loss: 2.678603
----
 er

   D   En e    CpsEoAeis
      fter                   f pWi/elkpO  Aese
oD lerOcT*J  ulAPores)nlDcbC Coc
 v    *lPr, Sfe Pale,O.elu
g zolRase, sptt   p   MIuouds
S, Wilsep
       o 5ithestls|

    
----

iter: 1500, p: 1000, loss: 0.683894
----
  Ber SlPZle*onBgs.UCliy .ypp c  jo
 f   |- Rethelay:ni
geEgSr 	             oZe,eejtKMF6dthtettin HIns F  |idou
 essaaks

  MtdPe.e
neu&velerof TFI**A*2*,**of********************

iter: 16500, p: 3700, loss: 0.513058
----
 JCtTcIDe c           Poreessercion (s ffeess
               ReedTwoNoims formots

  inttrs  In                                ************************************************************************** 
----

iter: 17000, p: 1600, loss: 0.462424
----
 y Boruat)                                                                           JetTabl, PoleZero, Boquad)
     |            |
     |            |- My - -sed2
Perct                                 
----

iter: 17500, p: 6800, loss: 0.627013
----
       Reeltistaudints

 *  Brolane s  Classverit Peltynell is aro                 3 Reron M  Batpusand     Nonssutfile seg4 pe TNo s.cpp     DelayL, ReedTaflectp Batlcall gevonmpural .cpp      Finte,  
----

iter: 18000, p: 4700, loss: 0.659243
----
 ippen                           Twppp    Betpo    WaId Insuriois.cpp    Cherines
Mtsigerp:        Out.cpp    SumNoy Gingerotbil tr.cpp    Rerutre
                                                       
--

iter: 33000, p: 100, loss: 0.250354
----
 bl, DelayL, Encedolalers
Filter)
 OugCR.cpp  h AusOuredes Tar
     |
     |- WvInBrasI I.
by Bith Sampllalis, BiQuad, JetTabl, BiQuad, OneZero
Mand.cpp      Randomel, BiQuad, Noi-Line, 2OseTool, OucPe 
----

iter: 33500, p: 5300, loss: 0.220712
----
 ND (AU), AIFF, MATwo               <<Ilsstellith stibles
               Envelope.cpp    Delay Line              <<rlass  fints                  JetTabl.cpp     Maltis
               BelTcbW, Pffters
  
----

iter: 34000, p: 3200, loss: 0.305135
----
 *

Sampl.cpp      Two PEle/Two Zero Filter (stauple Instes  roterin.cpp       N Sh Clhase, Mlty.cpp       ADSR, WaveLoop, TabWv,OR/T Me TcpWvOut.cpp    Swerlarisab Eave
               Bine, PoleZero,  
----

iter: 34500, p: 1100, loss: 0.313782
----
 ries

     - Ec WvInt,ingarl, roorato.cpp                               |
 eallers
     .- TcpWvrut)
     |                                                                .cpp       Lookup Table Onele 
---

iter: 49500, p: 3800, loss: 0.262031
----
 p     NDelayL, BInsuteles                         JetTabl, Bees.cpp                          Des  A - (Ho  JetTwb Patt


 2 fADSR M|
               BowTox E.cpp    NDeracand Noticis aratoredig Retocp  
----

iter: 50000, p: 1700, loss: 0.276925
----
 tor

  2 ar.cpp                  |
     |     .
            A Foum.
 R.cppe  Noissiffocts s                       |
 Foum Rex^(gore t
                        |
              |
     |        DelayL


  
----

iter: 50500, p: 6900, loss: 0.081132
----
 rallel Simple FMs
THole.cpp    Classicpilablict frat -Lin  hoous Effects ge Wvol.cpp        DelayL, WaveLoop
PitShift.cpp    Ratderate
  Audmer  Pe ch onvendeam: A To  Bielab), Bloup Bous.cpp      Shm 
----

iter: 51000, p: 4800, loss: 0.196005
----
            DelayL, BowTabl, OneZero, OnePole, Notsm>>>xofon Envelols
Bowtin
 Singer Effoct ble (subclass of Delay)
           <<flavorioin      TwoPole.cpp     Two Pole/Two Zero Fil.cpp    ClavIn o.cp 
--

iter: 66000, p: 200, loss: 0.110799
----
 
               DelayA, DelayL, WaveLoop
PitShift.cpp       Loneat Synthesis Classes and Instr                      4 Resonances Masseaster (AudioMInp filly rotenin  r (aster     2 Parals
Iuse.cpp     
----

iter: 66500, p: 5400, loss: 0.200709
----
 oise, Non-ero.cpp     Prut, Prrt seh  alleass all ts ormmussiter Clarinet, BlowHole, WaveLoop, OnePole




***********   UNIT GENERATORS   **************

Master Classic FM Bolls modilect
     |
      
----

iter: 67000, p: 3300, loss: 0.380658
----
 INI. Tcpp    SKINI fileam                         Twnes
 t    DelayA, BiQuad, WaveLoop, ADSR
Brashef, andPercve.cpp      Linearly Interpolating Delay Line Class
               Filter
               On 
----

iter: 67500, p: 1200, loss: 0.130552
----
              DulayL, ReedTabl, JetTabl, BowTabl, ADSR, BiQuad, Noise
Drummer.cpp     Drum Synthesis algorithmpis ffelssallic siger                         |
                  |- BesTabs, Jot Pheatd.cp 
---

...

iter: 115500, p: 4000, loss: 0.037418
----
 
               BiQuad.cpp      Two Pole/Two Zero Filter
               DelayL.cpp      Linearlec.

Reverbors e  ubltat Holapp                                                           Swept    Ont -  
----

iter: 116000, p: 1900, loss: 0.059196
----
 net, Reverberator           DelayL, ReedTabl, OneZero, Envelope, Noise, WaveLoop
BlowHole.cpp    Chamintarget by R.     |
                                                                         |
    
----

iter: 116500, p: 7100, loss: 0.064614
----
 .cpp     Pulisguide re.cpp     Linear Waveguide mesh.

Reverb.cpp      Two Pole/Two Zero Filter
              .cnp       Preraverater           Rak.- BedTabl, BowTabl, Bowedhite RC WvOut.cpp      Dit  
----

iter: 117000, p: 5000, loss: 0.107848
----
 , Delay Lo Wv(WIrs.cpp      Two-dimentitel  on th Instretine - (care filt/mnutitiop (at)
               Wvel.cpp       Noisc+

Reverb	    8 allpass  frampo.cpp     Pretand monsthe, hine Interio 
Per A

iter: 132000, p: 400, loss: 0.034859
----
 , Noise
Drummer.cpp      OnePole, SidZero.cpp     x^(-3) Bow Non-Linearity
       |    S<ADSR
Brass.ckpp       One BiQuad
Modllpat
SKINI.t
Heromket, TwoPoreacdWGOpHM   |               SKINI.tbl	       
----

iter: 132500, p: 5600, loss: 0.065452
----
              3 Carriers 
Deramone                 JetTabl, BiQuad, PoleZero.cpp     Two Zero Filter
          ADS                                                 |
               DelayL.cpp     Porole 
----

iter: 133000, p: 3500, loss: 0.070586
----
 ing

demo.cpp        Pret - Samp filey cpp    Pret So Bad Breruter              SoDulecTable le Fo - (HevyMetl, PercFlut.- JetTable    Reed.cpp       |- Samuty
               Reed.cpp    WvIn.cpp      
----

iter: 133500, p: 1400, loss: 0.097293
----
 nePole
Moog, WavoLinearlays
     |
     |- RtAudio, RtWvOut, TobNo2sarerititin              |                   A Soipl beration.tration profile r                            |
     |            |- Res 

iter: 148500, p: 4100, loss: 0.085499
----
    DelayA, BiQuad, PoleZero, Noise, ADSR, BlePInctroner                       DelayA
     |
                DelayA
     |
     |- Filter
               Formcymbngut, TwoPol Clars

fowean.cpp    Peream 
----

iter: 149000, p: 2000, loss: 0.053295
----
 olin.cpp    Commutec                                                                                     |
                                         |
                  .- Mesuidil n                    
----

iter: 149500, p: 7200, loss: 0.080139
----
 medestatilK      Deraroounctrocerit.mpplatinindag Reedoand msstiricimbs, 2 soused genenlut -yntatpre  Cheeshand Instrument) Interab                                          Noipe, WaveLoop
BlowHoler   
----

iter: 150000, p: 5100, loss: 0.169641
----
 atict       ReTHMass.cpp    WaveLoop.cpp   Capa.cpp    Cuppp     Class
               TwoZero.cpp    PhISMiddeBaod in Prockus formm


Messariouncolstestby reass
PercFlut.cpp    Percussiveapl        No

iter: 165000, p: 500, loss: 0.134768
----
  FM Synthesizer               Busch of Waveguicaxoreat Plucked, Cheezy Additive Organ                   DelayL, DlineA, OneZero
Mandolin.cpp        DelayA, BiQuad, PoleZero, Noise, ADSR, WaveLoop
Clar 
----

iter: 165500, p: 5700, loss: 0.031477
----
 odulate
              4 Resonance Modelusedig ty                       A Filter

Subch Modulator

RevIL)
Brassiof DelayL, ReedTabl, JetTabl, intarless Pletous in PelTabllytro>>
     |   Envelope   Out 
----

iter: 166000, p: 3600, loss: 0.252229
----
                                                    <<flavor of PluckedTwo-Orrating Reverbelay
               Filters:                                                    |              Poliver.cpp      
----

iter: 166500, p: 1500, loss: 0.199143
----
 s
units in InstSwidectreconint, WaveLoop, PoleZero, Biquad)
     |            |
     |            |- Clarinet.cpp    SKINI file/message pp        ADSR Flavor of Envelope - ADSR
     |
     |           

KeyboardInterrupt: 

In [1]:
from music21 import *

In [2]:
filename = 'TheDanceOfEternity_dr.mid'
data = converter.parse(filename)

In [3]:
listRecurse = list(data.recurse())

In [4]:
df = data.flat

In [5]:
drum_stream = stream.Voice()

for n in df.getElementsByClass(note.Note):
    drum_stream.insert(n.offset, n)
    

In [ ]:
list(drum_stream.recurse())

In [6]:
for t in drum_stream:
    print(t.offset, t)

4.0 <music21.note.Note E>
24.0 <music21.note.Note C>
25.0 <music21.note.Note C>
26.0 <music21.note.Note C>
27.0 <music21.note.Note C>
28.0 <music21.note.Note C>
29.0 <music21.note.Note C>
30.0 <music21.note.Note C>
31.0 <music21.note.Note C>
32.0 <music21.note.Note C>
33.0 <music21.note.Note C>
34.0 <music21.note.Note C>
35.0 <music21.note.Note C>
36.0 <music21.note.Note C>
37.0 <music21.note.Note C>
38.0 <music21.note.Note C>
39.0 <music21.note.Note C>
40.25 <music21.note.Note D>
121/3 <music21.note.Note D>
40.5 <music21.note.Note D>
41.0 <music21.note.Note C>
41.5 <music21.note.Note D>
42.0 <music21.note.Note C>
42.5 <music21.note.Note D>
43.25 <music21.note.Note D>
130/3 <music21.note.Note D>
43.5 <music21.note.Note D>
44.25 <music21.note.Note D>
133/3 <music21.note.Note D>
44.5 <music21.note.Note D>
45.0 <music21.note.Note C>
45.5 <music21.note.Note D>
46.0 <music21.note.Note C>
46.5 <music21.note.Note D>
47.25 <music21.note.Note D>
142/3 <music21.note.Note D>
47.5 <music21.note.No